In [1]:
from langchain.chains import FalkorDBQAChain
from langchain_community.graphs import FalkorDBGraph
from langchain_openai import ChatOpenAI

In [2]:
graph = FalkorDBGraph(database="movies")

In [3]:
graph.query(
    """
    CREATE 
        (al:Person {name: 'Al Pacino', birthDate: '1940-04-25'}),
        (robert:Person {name: 'Robert De Niro', birthDate: '1943-08-17'}),
        (tom:Person {name: 'Tom Cruise', birthDate: '1962-07-3'}),
        (val:Person {name: 'Val Kilmer', birthDate: '1959-12-31'}),
        (anthony:Person {name: 'Anthony Edwards', birthDate: '1962-7-19'}),
        (meg:Person {name: 'Meg Ryan', birthDate: '1961-11-19'}),

        (god1:Movie {title: 'The Godfather'}),
        (god2:Movie {title: 'The Godfather: Part II'}),
        (god3:Movie {title: 'The Godfather Coda: The Death of Michael Corleone'}),
        (top:Movie {title: 'Top Gun'}),

        (al)-[:ACTED_IN]->(god1),
        (al)-[:ACTED_IN]->(god2),
        (al)-[:ACTED_IN]->(god3),
        (robert)-[:ACTED_IN]->(god2),
        (tom)-[:ACTED_IN]->(top),
        (val)-[:ACTED_IN]->(top),
        (anthony)-[:ACTED_IN]->(top),
        (meg)-[:ACTED_IN]->(top)
"""
)

[]

In [21]:
graph.refresh_schema()
print(graph.schema)

import os

os.environ["OPENAI_API_KEY"] =os.getenv("OPENAI_API_KEY")

NameError: name 'graph' is not defined

In [6]:
chain = FalkorDBQAChain.from_llm(ChatOpenAI(temperature=0),allow_dangerous_requests = True, graph=graph, verbose=True)

In [7]:
chain.run("Who played in Top Gun?")

/var/folders/n2/2s0tjc891jx80m_1lwcfhm_40000gn/T/ipykernel_58647/1697278274.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Who played in Top Gun?")




> Entering new FalkorDBQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Top Gun'
RETURN p.name
Full Context:
[['Tom Cruise'], ['Val Kilmer'], ['Anthony Edwards'], ['Meg Ryan']]

> Finished chain.


'Tom Cruise, Val Kilmer, Anthony Edwards, Meg Ryan played in Top Gun.'

In [8]:
chain.run("Who is the oldest actor who played in The Godfather: Part II?")



> Entering new FalkorDBQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'The Godfather: Part II'
RETURN p.name, p.birthDate
ORDER BY p.birthDate ASC
LIMIT 1
Full Context:
[['Al Pacino', '1940-04-25']]

> Finished chain.


'Al Pacino is the oldest actor who played in The Godfather: Part II.'

In [9]:
chain.run("Robert De Niro played in which movies?")



> Entering new FalkorDBQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Robert De Niro'})-[:ACTED_IN]->(m:Movie)
RETURN m.title
Full Context:
[['The Godfather: Part II']]

> Finished chain.


'Robert De Niro played in The Godfather: Part II.'

In [2]:
graph1 = FalkorDBGraph(database="One-Hop to Pwn")

In [3]:
graph1.query(
    """
    MATCH (ip:IPRange {cidr: "0.0.0.0/0"})
        -[:INGRESS_TO]->(sg:SecurityGroup)
        <-[:ATTACHED_TO]-(vm:Compute)
        -[:ASSUMES]->(role:IAMRole)
        -[:ALLOWS]->(ds:DataStore)
    RETURN 
        ip.cidr AS Public_IP, 
        vm.name AS Exposed_VM, 
        role.name AS IAM_Role, 
        ds.name AS Accessible_DataStore
"""
)

[]

In [4]:
graph1.query(
    """
    CREATE (:IPRange {cidr: "0.0.0.0/0", name: "Internet"})
    CREATE (:Compute {name: "Web-Server-01", platform: "AWS", region: "us-east-1"})
    CREATE (:IAMRole {name: "DataAccessorRole"})
    CREATE (:DataStore {name: "Sensitive-Data-Bucket", type: "S3"})
    CREATE (:SecurityGroup {name: "PublicWebSG"})
    """
)

[]

In [5]:
graph1.query(
    """
    MATCH (i:IPRange {name: "Internet"}), (sg:SecurityGroup {name: "PublicWebSG"})
    CREATE (i)-[:INGRESS_TO]->(sg)
    """
)

[]

In [6]:
graph1.query(
    """
    MATCH (c:Compute {name: "Web-Server-01"}), (sg:SecurityGroup {name: "PublicWebSG"})
    CREATE (c)-[:ATTACHED_TO]->(sg)
    """
)


[]

In [7]:
graph1.query(
    """
    MATCH (c:Compute {name: "Web-Server-01"}), (r:IAMRole {name: "DataAccessorRole"})
    CREATE (c)-[:ASSUMES]->(r)
    """
)

[]

In [8]:
graph1.query(
    """
    MATCH (r:IAMRole {name: "DataAccessorRole"}), (ds:DataStore {name: "Sensitive-Data-Bucket"})
    CREATE (r)-[:ALLOWS {actions: "s3:GetObject"}]->(ds)
    """
)

[]

In [13]:
query_result = graph1.query(
    """
    MATCH (ip:IPRange {cidr: "0.0.0.0/0"})
        -[:INGRESS_TO]->(sg:SecurityGroup)
        <-[:ATTACHED_TO]-(vm:Compute)
        -[:ASSUMES]->(role:IAMRole)
        -[:ALLOWS]->(ds:DataStore)
    RETURN 
        ip.cidr AS Public_IP, 
        vm.name AS Exposed_VM, 
        role.name AS IAM_Role, 
        ds.name AS Accessible_DataStore
"""
)


In [15]:
# Create a new Security Group named "SSH-Access"
graph1.query(
    """
    CREATE (:SecurityGroup {name: "SSH-Access"})
    """
)

# Create a relationship from the Internet to this new Security Group,
# with a property that specifies the port.
graph1.query(
    """
    MATCH (i:IPRange {name: "Internet"}), (sg:SecurityGroup {name: "SSH-Access"})
    CREATE (i)-[:INGRESS_TO {port: 22}]->(sg)
    """
)

# Attach the existing Web-Server-01 to this new security group
graph1.query(
    """
    MATCH (c:Compute {name: "Web-Server-01"}), (sg:SecurityGroup {name: "SSH-Access"})
    CREATE (c)-[:ATTACHED_TO]->(sg)
    """
)

[]

In [16]:
print(query_result)


[['0.0.0.0/0', 'Web-Server-01', 'DataAccessorRole', 'Sensitive-Data-Bucket']]


In [22]:
graph1.refresh_schema()
print(graph1.schema)

import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

Node properties: [[OrderedDict({'label': 'DataStore', 'keys': ['name', 'type']})], [OrderedDict({'label': 'IAMRole', 'keys': ['name']})], [OrderedDict({'label': 'IPRange', 'keys': ['cidr', 'name']})], [OrderedDict({'label': 'Compute', 'keys': ['name', 'platform', 'region']})], [OrderedDict({'label': 'SecurityGroup', 'keys': ['name']})]]
Relationships properties: [[OrderedDict({'types': 'INGRESS_TO', 'keys': ['port']})], [OrderedDict({'types': 'ALLOWS', 'keys': ['actions']})], [OrderedDict({'types': 'ASSUMES', 'keys': []})], [OrderedDict({'types': 'ATTACHED_TO', 'keys': []})]]
Relationships: [[OrderedDict({'end': 'SecurityGroup', 'start': 'IPRange', 'type': 'INGRESS_TO'})], [OrderedDict({'end': 'IAMRole', 'start': 'Compute', 'type': 'ASSUMES'})], [OrderedDict({'end': 'SecurityGroup', 'start': 'Compute', 'type': 'ATTACHED_TO'})], [OrderedDict({'end': 'DataStore', 'start': 'IAMRole', 'type': 'ALLOWS'})]]



In [18]:
chain1 = FalkorDBQAChain.from_llm(ChatOpenAI(temperature=0),allow_dangerous_requests = True, graph=graph1, verbose=True)

In [20]:
chain1.run("What security groups are exposed to the internet on port 22?")



> Entering new FalkorDBQAChain chain...
Generated Cypher:
MATCH (sg:SecurityGroup)-[r:INGRESS_TO]->(ip:IPRange)
WHERE r.port = 22
RETURN sg
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [19]:
chain1.run("Flag any SSH exposure")




> Entering new FalkorDBQAChain chain...
Generated Cypher:
MATCH (c:Compute)-[:ATTACHED_TO]->(sg:SecurityGroup)-[:INGRESS_TO]->(ip:IPRange)
WHERE sg.name = 'SSH'
RETURN c, sg, ip
Full Context:
[]

> Finished chain.


"I don't know the answer."